# DVS Set To Work

* Wideband RF Envelope Check
* Signal Path Gains
* Pointing Models
* Delay Models
* Antenna Efficiency
* Feed Indexer Angles

                                                                Last updated 11/02/2026

In [ ]:
%matplotlib inline
import pylab as plt
import numpy as np

In [ ]:
from dvs import cattools, planning, wideband, fastgain, tracking, hologreport, util
from analysis import katseops, katsepnt, katselib, fit_pointing_model
import katpoint

In [ ]:
katselib.initsensorcache() # Init the local cache that speeds up katselib.getsensordata()

In [ ]:
ant = "e121"

## Initial Delay Model


In [ ]:
llh, enu_niao = cattools.nominal_pos("SKA121", verbose=True)

## Wideband Envelope Measurements

### Band2

In [ ]:
T_meas = katpoint.Timestamp("2025-04-02 11:58").secs
b2_sn = katselib.getsensorvalues(ant+"_spfc_serialNumbers_2", [T_meas-24*3600,T_meas], interpolate=None)[1][-1]

## If you need to remove RFI, uncomment the following and then use process_wbg_set(dataset, ...) instead of filename
# dataset = wideband.WBGDataset.load("./l1_data/WBG8GHz_SKA121_02042025")
# dataset.on, dataset.off = util.remove_RFI(dataset.freq, dataset.on, dataset.off, "../catalogues/rfi_mask.txt", flag_thresh=0.002)
ds = wideband.process_wbg_set("./l1_data/WBG8GHz_SKA121_02042025", "SKA_B2", flim=(0,4e9), figsize=(12,10))
plt.suptitle("WBG8GHz_SKA121_02042025 | SPFB2 SN-"+b2_sn[-3:]);
plt.gcf().savefig("WBG8GHz_SKA121_02042025.png", dpi=300, bbox_inches="tight")

In [ ]:
planning.radiosky(T_meas, tabulate=False, f_MHz=1400, catfn="../catalogues/sources_all.csv", el_limit_deg=5)
planning.hp.graticule()
# Overplot ~SCP on the orthographic view
gl, gb = katpoint.sphere_to_plane["SIN"](0,np.pi/2, np.radians(180)-np.pi,np.radians(31)) # Ortho projection for az,el=180,31
plt.gcf().axes[0].plot(gl,gb, 'yX', markersize=20);

# Radiometer Assessment

### Ku-band

NB: need to adjust DVS TFR Ku LO power level to minimum that HV spectrum does not show drop-out in upper part of band, with the full range of LO frequencies.
For MKE121: use 10dB.

In [ ]:
rr = katselib.ls_archive(f"Antennas:*{ant}* AND StartTime:[2024-11-26T15:0:0Z TO *] AND Description:Ku AND Radiometer AND InstructionSet:*track*",
                         fields=["CaptureBlockId","StartTime","CenterFrequency","Description","Targets"])

In [ ]:
# Most of the above are affected by low-level intermittent drop-outs between digitiser & correlator.
# Some of these have issues with blocks of channels - data lost. Malfunction or temporary data staging issue?

cbids = [1732716601,
         1732739243,
         ]

cbid = cbids[0]
ds = util.open_dataset(cbid, ref_ant=ant, cache_root="./l1_data")
fastgain.analyse(ds, ant);

In [ ]:
cbid = cbids[1]
ds = util.open_dataset(cbid, ref_ant=ant, cache_root="./l1_data")
fastgain.analyse(ds, ant);

### S-band

In [ ]:
rr = katselib.ls_archive(f"Antennas:*{ant}* AND StartTime:[2024-12-01T0:0:0Z TO *] AND Description:S AND (Radiometer OR Short) AND InstructionSet:*track*",
                         fields=["CaptureBlockId","StartTime","CenterFrequency","Description","Targets"])

In [ ]:
# Different requant gains
cbids = [1733311557, # 3234. A few samples lost, but accurately flagged?
         1733314056, # 500. The last sample is half-bad but not flagged out
         1733315245, # EQ gain way too low
         1733316341, # 20000. Some data lost & flagged
         1737896107] # 1k. Poor

cbid = cbids[0]
ds = util.open_dataset(cbid, ref_ant=ant, cache_root="./l1_data")
fastgain.analyse(ds, ant);

In [ ]:
cbid = cbids[1] # The last sample is half-bad but not flagged out - screws with AVAR plot?
ds = util.open_dataset(cbid, ref_ant=ant, cache_root="./l1_data")
fastgain.analyse(ds, ant, timerange=(ds.timestamps[0],ds.timestamps[-2]));

In [ ]:
cbid = cbids[2] # EQ gain way too low
ds = util.open_dataset(cbid, ref_ant=ant, cache_root="./l1_data")
fastgain.analyse(ds, ant);

In [ ]:
cbid = cbids[3] # EQ gain way too high, some data lost & flagged
ds = util.open_dataset(cbid, ref_ant=ant, cache_root="./l1_data")
fastgain.analyse(ds, ant, xK=[1.2]); # Only 1/98 chunk passes at 1.1, 2/98 at 1.15, 3/98 at 1.2!

In [ ]:
cbid = cbids[4] # 1k with gain too low?
ds = util.open_dataset(cbid, ref_ant=ant, cache_root="./l1_data")
fastgain.analyse(ds, ant, xK=[1.05,1.1,1.2]);

## Band2

In [ ]:
rr = katselib.ls_archive(f"Antennas:*{ant}* AND StartTime:[2024-12-01T0:0:0Z TO *] AND Description:Band2 (Short OR Flatness) AND InstructionSet:*track*",
                         fields=["CaptureBlockId","StartTime","CenterFrequency","Description","Targets"])

In [ ]:
cbids = [1739622081, # Horrible - Sun in sidelobes!??? 
         1739641121] # 1k. Poor

cbid = cbids[0]
ds = util.open_dataset(cbid, ref_ant=ant, cache_root="./l1_data")
fastgain.analyse(ds, ant, xK=[1.05,1.1,1.3,1.5]);

In [ ]:
cbid = cbids[1]
ds = util.open_dataset(cbid, ref_ant=ant, cache_root="./l1_data")
fastgain.analyse(ds, ant, xK=[1.05,1.1,1.3,1.5,1.8]);

# Pointing Models

For set-to-work we use wide, single dish raster scans to set up the pointing models. Later, when we have 2 or more antennas, with reasonable pointing & delay tracking, we only use "interferometric circular scans"!

In [ ]:
NIGHT = [("sun_el",lambda e:e<=0)]
DAY = [("sun_el",lambda e:e>0)]
# Environmental conditions as per SKA-TEL-DSH-0000005 rev 6B and later
PRECISION = [('wind_speed',lambda w:w<=5), ('wind_dynamic',lambda w:w<=3*0.7)]
UPTO_STANDARD = [('wind_speed',lambda w:w<=7), ('wind_dynamic',lambda w:w<=3*1.0)] # PRECISION+STANDARD

best_pm = {}

## Collect the Data

### Check for Alignment of Activity Flags

In [ ]:
# ONCE-OFF! For the first raster scan on a new kind of DishProxy, check that the partitioning of activities are correct.
# For a new DishProxy we may update util.open_dataset() to compensate for any misalignment - typically due to different COMMAND_LEAD_TIMEs.
# If necessary, update util.open_dataset() to get the partitioning to be correct.
#import importlib; importlib.reload(util)

cbid = 1745570892
katselib.plot_pointing(util.open_dataset(cbid, ref_ant=ant), ants=ant)

In [ ]:
# For reference: what does the partitioning look like without special compensation?
import katdal
katselib.plot_pointing(katdal.open(util.cbid2url(cbid)), ants=ant)

In [ ]:
# Beacon frequencies for GEOS [MHz]
geo_bcn = {"APSTAR-7":[(11699.7,11699.9), (12250.9,12251.1)],
           "INTELSAT 22 (IS-22)":[(11698.9,11699.1), (12498.9,12499.1)],
           "INTELSAT 17 (IS-17)":[(11197.9,11198.1), (11451.9,11452.1), (12501.9,12501.1)],
           "YAMAL 402":[(11459.4,11459.6), (12499.9,12500.1)],
           "AZERSPACE 2 (IS-38)":[(11702.9,11703.1), (12746.9,12747.1)], # "Double shape" - rather avoid
           "EUTELSAT 36D":[(11701.7,11701.9), (12500.9,12501.1)], # WARNING! EUTELSAT 36D and neighbouring 36C beacons are separated by only 0.5MHz!
           "EUTELSAT 21B":[(11200.1,11200.3), (11699.7,11699.9), (12499.9,12500.1), (12500.4,12500.6)],
           "ASTRA 1P (SES-24)":[(11448.9,11449.1)], # (11199.9,11200.1)
           "EUTELSAT 10B":[(11699.1,11699)], # (11698.5,11698.7)
           "EUTELSAT 9B":[(11701.1,11701.3), (11703.3,11703.5), (12497.9,12498.1)],
           "THOR 7":[(11705.9,11706.1), (12494.4,12495.6)],
           "INTELSAT 10-02":[(11197.9,11198.1), (11451.9,11452.1)], # (11200.9,11201.1)
           "EUTELSAT 7 WEST A":[(11199.4,11199.6), (11200.4,11200.6), (12497.9,12499.1)],
           "EUTELSAT 8 WEST B":[(11199.4,11199.6), (12500.9,12501.1)], # (11200.4,11200.6)
           "INTELSAT 37E (IS-37E)":[(11198.9,11199.1), (11451.4,11451.6),(12500.9,12501.1)],
           "SES-4":[(11450.9,11451.1), (12501.9,12502.1)],
           "INTELSAT 904 (IS-904)":[(11451.9,11452.1)], # (11197.9,11198.1)
           "INTELSAT 28 (IS-28)":[(11451.9,11452.1)],
           "HISPASAT 36W-1":[(11451.9,11452.1), (12748.9,12749.1)],
          }
def frng2mask(frng, freqs): # Create boolean mask where freqs overlaps frng=(fstart, fstop)
    df = (freqs[1]-freqs[0])/2
    matches = np.sum([(f[0]-df<=freqs) & (freqs<=f[-1]+df) for f in frng], axis=0).astype(bool)
    if (np.count_nonzero(matches) == 0): # No beacons in band: relaxed or strict
        matches = np.full(freqs.shape, True) # Entire range
        # matches = (freqs <= freqs[0]) # Just DC channel - which gets flagged and results in this record being skipped by reduce_pointing_scans
    return matches

In [ ]:
## Bootstrap Raster Scans
# Rasters on GEOS - e.g. when starting from SCRATCH in Ku-band
rr = katselib.ls_archive(f"Antennas:*{ant}* AND StartTime:[2026-01-08T08:0:0Z TO *] AND Description:GEOS AND InstructionSet:*point_source_scan*",
                         min_duration=900, fields=["CaptureBlockId","StartTime","CenterFrequency","Description","Targets","Antennas"])
print()
# Other rasters
rr = katselib.ls_archive(f"Antennas:*{ant}*  AND StartTime:[2026-01-08T08:0:0Z TO *] AND Description:Pointing AND InstructionSet:*point_source_scan*",
                         min_duration=900, fields=["CaptureBlockId","StartTime","CenterFrequency","Description","Targets","Antennas"])

In [ ]:
# These are all typically with initial FI angles
b2_cbids = []

s_cbids = []

ku_cbids = []

In [ ]:
## Fit centroids to "single dish" datasets, with debug plots
cbids = ku_cbids

for cbid in cbids:
    ds = util.open_dataset(cbid, ref_ant=ant)
    rfi_mask = util.load_rfi_static_mask("../catalogues/rfi_mask.txt", ds.freqs)
    rfi_mask |= (ds.channels < 50) | (len(ds.channels)-50 < ds.channels) # Tune this if necessary
    freqsMHz = ds.freqs/1e6
    chans = lambda tgt,_: frng2mask(geo_bcn[tgt], freqsMHz) if (tgt in geo_bcn.keys()) else ~rfi_mask
    tracking.reduce_pointing_scans(ds, ant, track_ant=None, kind='raster', scans="scan", chans=chans, strict=True, 
                                   output_filepattern="./l2_data/%s_%s_point_source_scans.csv", debug=True, verbose=True,
                                   clip_radius=1.5) # Tune this to eliminate strong neighbours, or ND flags if present

In [ ]:
## Proper Pointing Measurements
rr = katselib.ls_archive("StartTime:[2026-01-09T0:0:0Z TO *] AND Description:All Sky Pointing AND InstructionSet:*circular_pointing*" +
                         f" AND InstructionSet:\"scan-ant {ant}\"~1", min_duration=900, fields=["CaptureBlockId","StartTime","CenterFrequency","Description","InstructionSet"], field_len=220)

In [ ]:
b2_ccbids = [] # Initial FI angle

s_ccbids = [] # Initial FI angle

ku_ccbids = [] # Initial FI angle

In [ ]:
## Fit centroids to "interferometric" datasets, with debug plots
cbids = b2_ccbids

for cbid in cbids:
    ds = util.open_dataset(cbid, ref_ant=ant, cache_root="./l1_data")
    rfi_mask = util.load_rfi_static_mask("../catalogues/rfi_mask.txt", ds.freqs)
    rfi_mask |= (3000e6 < ds.freqs) & (ds.freqs <=3500e6) # Prefer to not fit this part of S-band, due to beam features
    for track_ant in ds.obs_params['track_ants'].split(','):
        try:
            tracking.reduce_pointing_scans(ds, ant, track_ant=track_ant, kind='cardioid', scans="track", chans=~rfi_mask, strict=True,
                                           phased_up=np.max(ds.freqs)<4e9, # True only for B2 & S-band!
                                           output_filepattern=f"./l2_data/%s_%s_{track_ant}_circular_pointing.csv", debug=True, verbose=True)
        except Exception as e: # Likely not correct track_ant
            print("INFO: unable to process %s for %s_%s" % (cbid, ant, track_ant), e)

In [ ]:
!ls -la ./l2_data

In [ ]:
# Check hardware status over the time span of measurements
cbids = b2_cbids

data1, _ = katsepnt.load_apss_data([katsepnt.find_files(f"{cbid}_*l0_{ant}*.csv", "./l2_data")[0] for cbid in cbids])
# FI angles, directly from sensor database
katselib.plot_sensors(data1['timestamp'], [ant+"_dsm_indexerActualPosition"], interpolate="zero", figsize=(12,3))
# Tiltmeter corrections, directly from sensor database
katselib.plot_sensors(data1['timestamp'], [ant+"_dsm_tiltPointCorrEnabled"], interpolate="zero", figsize=(12,3))

In [ ]:
# Inspect pointing data from each dataset "as-is"
figs = [] # Collect figures to overplot the datasets
for cbid in sorted(cbids):
    S = katsepnt.eval_pointingstability([f"{cbid}*l0_{ant}*.csv"], root="./l2_data",
                                        filters=[], update_model=False,
                                        metrics=["timestamp","azimuth","elevation","snr_I","wind_speed","sun_el"], figs=figs)
    print(cbid, len(S.TS))

In [ ]:
katsepnt.plot_skycover(cbids, ant, "./l2_data")

## Construct Pointing Models

You may use any of the columns in the csv file as a "metric".

If there are too few (good) points, only fit terms that may be significant!
* The azimuth zero (P1), axis orthogonality (P3) and also pedestal vertical (P5 & P6) should be the same for Optical & RF. For MKE121 it is possible that the Optical Model is "no good at all".
* The gravity terms (P8 & P11) will be different between optical and RF, and also the feed alignment terms (P4 cross-elevation & P7 vertical).
* Each feed band should have different alighment terms (P4 cross-elevation & P7 vertical).

### BOOTSTAP Model from RASTERs
IF we have to start a model from scratch, we typically use raster scan data and just fit the minimum terms (e.g. P1, P7) to get us to < 120arcsecRMS.

This can be done in any band.

In [ ]:
# Zero'th order model
cbids = s_cbids
print(cbids)

In [ ]:
# Explore to see which terms are significant, possibly which targets or points to omit?
# Note: there's typically insufficient sky coverage at this stage to fit any of P3,P5,P6,P8,P11.
figs = []
for pct in [100]:
    katsepnt.fit_pointingmodels(ant, cbids, "./l2_data/", fnpattern="%(cbid)s*l0_%(ant)s*.csv",
          filter0=[], outlier_pct=pct,fit_for_outliers=True,filter1=[],
          statistic='MAV', as_is=False, P_fits=[[1,7],[1,4,7]], figs=figs)
    print("-"*60)

In [ ]:
## Check after model was updated
cbids = s_cbids # We may combine datasets with different pointing models - as long as the "physical state" didn't change!
print(cbids)

In [ ]:
figs = []
for pct in [100]:
    katsepnt.fit_pointingmodels(ant, cbids, "./l2_data/", fnpattern="%(cbid)s*l0_%(ant)s*.csv",
          filter0=[], outlier_pct=pct,fit_for_outliers=True,filter1=[],
          statistic='MAV', as_is=False, P_fits=[[1,7],[1,4,7]], figs=figs)
    print("-"*60)

In [ ]:
# Stop if there's convergence, and RMS < 120arcsec

### S-band
Initial model is based on final BOOTSTAP model.

In [ ]:
cbids = s_ccbids
print(cbids)

outlier_targets = katsepnt.find_outlier_targets(cbids, ant, outlier_pct=99.5, outlier_count=2, debug=True)
#outlier_targets = ['J0519-4546'] # Modify this list if judged necessary after further investigation!
BAD_TGT = [("target", lambda t:t not in outlier_targets)]

In [ ]:
figs = []
for pct in [85,75,65,55]: 
    katsepnt.fit_pointingmodels(ant, cbids, "./l2_data/", fnpattern="%(cbid)s*l0_*%(ant)s*.csv",
          filter0=BAD_TGT,outlier_pct=pct,fit_for_outliers=True, filter1=[],
          statistic='MAV', as_is=False, P_fits=[[1,4,5,6,7]], figs=figs)

In [ ]:
# Add P3 & P11 -> check for improvement in redisuals & convergence
figs = None
for pct in [85,75,65,55]: 
    katsepnt.fit_pointingmodels(ant, cbids, "./l2_data/", fnpattern="%(cbid)s*l0_*%(ant)s*.csv",
          filter0=BAD_TGT,outlier_pct=pct,fit_for_outliers=True, filter1=[], 
          statistic='MAV', as_is=False, P_fits=[[1,3,4,5,6,7,11]], figs=figs)

In [ ]:
# Add P8 -> check for improvement in redisuals & convergence
figs = None
for pct in [85,75,65,55]: 
    katsepnt.fit_pointingmodels(ant, cbids, "./l2_data/", fnpattern="%(cbid)s*l0_*%(ant)s*.csv",
          filter0=BAD_TGT,outlier_pct=pct,fit_for_outliers=True, filter1=[], 
          statistic='MAV', as_is=False, P_fits=[[1,4,5,6,7,8]], figs=figs)

In [ ]:
# "Optimal" model from above
best_pm[ant] = katsepnt.fit_pointingmodels(ant, cbids, "./l2_data/", fnpattern="%(cbid)s*l0_*%(ant)s*.csv",
                                      filter0=BAD_TGT,outlier_pct=75,fit_for_outliers=True, filter1=[], 
                                      statistic='MAV', as_is=False, P_fits=[[1,3,4,5,6,7,8,11]], figs=None)[0]

# Show residuals from ALL points with this model
figs = dict(metrics=["azimuth","elevation","sun_el","wind_speed"], figs=[])
S = katsepnt.eval_pointingstability([f"{cbid}*l0_{ant}*.csv" for cbid in cbids], root="./l2_data", apply_pm=best_pm,update_model=False, filters=[], **figs)
katsepnt.plot_pointingresiduals(S.de, S.dxe, ant, outlier_pct=[100, 95])

### Ku-band
Initial model is based on final BOOTSTAP model.

### Band2
Initial model is based on final BOOTSTAP model.

# Delay Model On-sky

Some notes:
* All bands must have the same E,N,U values - so use the band with best quality data, e.g. S-band.
* Particularly for Ku-band: GEOS datasets are usable but only for rough East & cable length terms.
* PREFERE 4k for these measurements (significantly reduces "ambiguity interval" over 1k)

In [ ]:
def fit_delaymodels(cbids, ref_ant, args, pols="H,V", make_plots=True, rfi_mask="../catalogues/rfi_mask.txt"):
    """@param cbids: datasets comprised of 'track' on targets, all must have exactly the same telescope configuration!
                   NB: Can only combine datasets if center frequency, number of channels and dump rate are the same.
                       ALSO only if delay model is the same (no error message - just silently skips all but the first one).
       @param args: text, typically "-f 0,4096 -s 0.9" may also add " --discard-tracklength 20" to drop tracks with unidentified hang/crash
       @return: {pol:([ants, enu, sigma_enu, old_enu, cable_len, sigma_c_l, old_c_l, niao, sigma_niao, old_niao])} for all polarisations
    """
    if make_plots: args += " --make-plots"
    if rfi_mask: args += " --rfi-mask %s"%rfi_mask
    
    for cbid in cbids: # Ensure datasets are cached
        util.open_dataset(cbid, cache_root="./l1_data")
    _results = {}
    gg = dict(globals()) # We use %run to have figures displayed, but that messes with globals. So make a copy of globals, to restore afterwards
    for pol in pols.split(","):
        %run ../bin/baseline_cal_reduction.py {" ".join([f"./l1_data/{cbid}/{cbid}_sdp_l0.full.rdb" for cbid in cbids])} -r {ref_ant} -p {pol} {args}
        _results[pol] = ([a.name for a in data.ants], positions, sigma_positions, old_positions, cable_lengths, sigma_cable_lengths, old_cable_lengths, niao, sigma_niao, old_niao)
    GG = globals()
    for k,v in gg.items():
        GG[k] = v
    
    return _results

def plot_delay_convergence(rr, ant):
    """ @param rr: list of {'H':(antnames, ...),'V':(antnames, ...)} as returned by fit_delaymodels()
        @param ant: name of antenna for which to plot convergence results
        @return: mean values of rr (also averaged over polarisations, except for cable lengths), in same format as fit_delaymodels() """
    pols = sorted(rr[0].keys())
    r_by_pol = [[] for p in pols] # Results only for this ant
    for r in rr:
        ant_idx = r[pols[0]][0].index(ant)
        for i,p in enumerate(pols):
            r_by_pol[i].append([_[ant_idx] for _ in r[p][1:]]) # Exclude antnames
    
    axs = plt.subplots(4,2, width_ratios=[3,1], sharex='col', figsize=(12,8))[1]
    # Series of ENUC along with 3*sigma error bars
    for rrp,fmt in zip(r_by_pol, "_./\\"):
        for i,r in enumerate(rrp):
            for j,p in enumerate("ENU"):
                axs[j][0].errorbar(i, r[0][j], 3*r[1][j], fmt=fmt, capsize=3)
            axs[3][0].errorbar(i, r[3], 3*r[4], fmt=fmt, capsize=3)
        for j,p in enumerate("ENU"):
            axs[j][0].hlines(r[2][j], 0,len(rrp)-1, 'k'); axs[j][0].set_ylabel(p)
        axs[3][0].hlines(r[5], 0,len(rrp)-1, 'k'); axs[3][0].set_ylabel("C")
        for j in range(4):
            axs[j][0].grid(True)

    # Histograms of ENUC
    for j,p in enumerate("ENU"):
        values = [r[0][j] for r in (r_by_pol[0]+r_by_pol[1])] # All pols together
        axs[j][1].hist(values, orientation='horizontal'); axs[j][1].legend(["~%.2f"%np.mean(values)])
    for p,rrp in zip(pols,r_by_pol): # This is unique by pol
        values = [r[3] for r in rrp]
        axs[3][1].hist(values, orientation='horizontal', label="%s ~ %.2f"%(p,np.mean(values)))
    axs[3][1].legend()
    
    # Calculate the mean values for all antennas - like what's done for histogram plot above
    # Keep the same output format as fit_delaymodels(), just replace results for ant by averages
    import copy
    enuc_mean = {pols[0]:[copy.copy(_) for _ in rr[0][pols[0]]], 
                 pols[1]:[copy.copy(_) for _ in rr[0][pols[1]]]}
    for ant in enuc_mean[pols[0]][0]:
        r_by_pol = [[] for p in pols] # Results only for this ant
        for r in rr:
            ant_idx = r[pols[0]][0].index(ant)
            for i,p in enumerate(pols):
                r_by_pol[i].append([_[ant_idx] for _ in r[p][1:]]) # Exclude antnames
        for j,p in enumerate("ENU"):
            values = [r[0][j] for r in (r_by_pol[0]+r_by_pol[1])] # All pols together
            enuc_mean[pols[0]][1][ant_idx][j] = enuc_mean[pols[1]][1][ant_idx][j] = np.mean(values)
        for p,rrp in zip(pols,r_by_pol): # This is unique by pol
            values = [r[3] for r in rrp]
            enuc_mean[p][4][ant_idx] = np.mean(values)
    return enuc_mean

def calc_delaymodel_delta(r, ant, ref_ant=None):
    """ Calculate and print Deltas for ant vs. ref_ant, from results generated by fit_delaymodels() """
    for p,rp in r.items():
        ants, enu, c = rp[0], rp[1], rp[4] # cf. fit_delaymodels()
        ant_ix = ants.index(ant)
        ref_ix = sorted(set(range(len(ants))) - {ant_ix})
        ref_ix = ants.index(ref_ant) if (ref_ant) else ref_ix[0]
        D_enu = np.r_[enu[ant_ix]] - np.r_[enu[ref_ix]]
        D_enuc = np.concatenate([D_enu, [c[ant_ix] - c[ref_ix]]])
        print(p, ants[ant_ix],ants[ref_ix], D_enuc)

In [ ]:
rr = katselib.ls_archive(f"Antennas:*{ant}* AND StartTime:[2025-04-25T0:0:0Z TO *] AND Description:DelayCal Track AND InstructionSet:*track*",
                         min_duration=5, fields=["CaptureBlockId","StartTime","CenterFrequency",'Description','Targets','Antennas'])

# Note: can only combine datasets if center frequency, number of channels and dump rate are the same.
# ALSO only if delay model is the same (no error message - just silently skips all ut the first one)

## S-band
These datasets typically converge nicely i.e condition number and error bar shrink as -s is increased.

In [ ]:
# 4k@1sec, S4!
fit_delaymodels([1745625709], "s0121", "-f 10,1014 -s .9")

In [ ]:
## -> Perform sensitivity analysis, if deemed necessary

rr = [fit_delaymodels([1745625709], "s0121", "-f 20,1000 -s %.2f"%s, make_plots=False) \
           for s in np.arange(.6,1+0.02,0.1)]

In [ ]:
plot_delay_convergence(rr, ant)

In [ ]:
# -> use -s 0.8; now remove channels at low end
rr = [fit_delaymodels([1745625709], "s0121", "-f %d,1024 -s .8"%f) \
           for f in range(0,200,20)]

In [ ]:
plot_delay_convergence(rr, ant)

In [ ]:
# -> use -f 20,...; now remove channels at low end
rr = [fit_delaymodels([1745625709], "s0121", "-f 20,%d -s .8"%(1024-f), make_plots=False) \
            for f in range(0,200,20)]

In [ ]:
plot_delay_convergence(rr, ant)

## Band2

In [ ]:
# 4k@1sec
fit_delaymodels([1752077526], "s0121", "-f 10,1014 -s .9")

## Ku-band

Ku-band fringes are typically weak.

Approach (if the Ku-band time-shares the same digitiser as Band2):
- keep ENU fixed as per Band2 solution!

In [ ]:
# 4k@1sec; poor result, consider sensitivity analysis!
fit_delaymodels([1752077526], "s0121", "-f 10,1014 -s %.2f"%s)

In [ ]:
# Evaluate different outlier thresholds.
rr = [fit_delaymodels([1752077526], "s0121", "-f 10,1014 -s %.2f"%s, make_plots=False) \
       for s in np.arange(.6,1+0.02,0.2)]

In [ ]:
plot_delay_convergence(rr, ant)

In [ ]:
# -> use -s <= 0.80; now remove channels from the low end
rr = [fit_delaymodels([1752077526], "s0121", "-f %d,1024 -s .80"%f, make_plots=False) \
           for f in range(0,300,40)]

In [ ]:
plot_delay_convergence(rr, ant)

In [ ]:
# -> use -f 40,...; now remove channels from the high end
rr = [fit_delaymodels([1752077526], "s0121", "-f 40,%d -s .80"%(1024-f), make_plots=False) \
           for f in range(0,300,40)]

In [ ]:
plot_delay_convergence(rr, ant)

# Holography
Preliminary holography, just to allow Feed Indexer angles to be confirmed.

In [ ]:
# This seems more appropriate for showing 'devmaps' than the default offered by katholog.
CM = plt.cm.viridis

F_mr, f_eq = 5.8535, 8.507 # Derived from SKA-TEL-DSH-0000018 rev 2 & DS Design Report 316-000000-022 - EM Sensitivity Analysis

# PREDICTED patterns generated rel. to secondary focus (F0 in SKA-TEL-DSH-0000018 rev 2), shift to center on Q0
pDISHPARAMS = dict(telescope="SKA", xyzoffsets=[0.0, 1.476396+8.04, 0], xmag=-f_eq/F_mr, focallength=F_mr)

# Parameters for MEASURED patterns
DISHPARAMS = dict(telescope="SKA", xyzoffsets=[0.0, 1.5363, 0.2330], xmag=-f_eq/F_mr, focallength=F_mr)

# Tilt angles calculated from geo orbital slots, extra tilt and relative to MeerKAT center
geotilt = {"EUTELSAT 36D": (36.1,+3.535), # Extra +3.535 tilt for most Eutelsat
           "BADR-7": (26.0,0),
           "EUTELSAT 21B": (21.6,+3.535),
           "EUTELSAT 10B": (10.0,+3.535),
           "EUTELSAT 9B": (9.0,+3.535),
           "INTELSAT 37E (IS-37E)": (-18,0),
           "INTELSAT 25 (IS-25)": (-31.5,0),
         }
linpol = lambda pol, satellite_ID: hologreport.e_bn(pol, geotilt[satellite_ID])

predL = None
predS = None
predK = hologreport.ResultSet("predicted", f_MHz=[], beacon_pol=[], clipextent=2.8) # Max extent 2.8deg

In [ ]:
# Predicted maps for Ku-band
for f_MHz,pol in [(11452,"RCP"),(11700,"RCP"),(11700,"LCP"),(12501.5,"RCP")]:
    b, aH, aV = hologreport.load_predicted(f_MHz, pol, pDISHPARAMS, el_deg=45, clipextent=predK.clipextent, gridsize=512)
    predK.f_MHz.append(f_MHz); predK.beacon_pol.append(pol)
    predK.beams.append(b); predK.apmapsH.append(aH); predK.apmapsV.append(aV)

In [ ]:
rr = katselib.ls_archive(f"InstructionSet:\"scan-ant*{ant}*\" AND StartTime:[2025-01-20T0:0:0Z TO *] AND InstructionSet:*holography*",
                    min_duration=1800, fields=["CaptureBlockId","StartTime","CenterFrequency",'Description','InstructionSet'], field_len=250)

In [ ]:
# NOTE1: add tag "extralmoffset=auto" when there are >1 scan ants
# NOTE2: add tag "findwrap" if the 


# With FI @ 103.28deg
ms_Ku0 = [
    hologreport.ResultSet(1745942382,f_MHz=[12499],beacon_pol=["LCP"],clipextent=6,tags=["FIk0","day","IS 22"]),
    hologreport.ResultSet(1746149694,f_MHz=[12499],beacon_pol=["LCP"],clipextent=6,tags=["FIk0","night","IS 22"]),
]

# With FI @ -18.62deg
ms_Sband0 = [ # To be loaded with dMHz=30 (1% fractional bandwidth)
     hologreport.ResultSet(1745972305,f_MHz=[2700,3300,3401],beacon_pol=None,clipextent=9,tags=["FIs0","night","J1924-2914"]),
]

In [ ]:
# Check for channels, bandwidth & polswap.
# CAUTION: this often gets the signs of RCP/LCP wrong!
for ms in ms_Ku0:
    ds = util.open_dataset(ms.fid, ref_ant=ant)
    katselib.plot_signalpol(ds, f_MHz=ms.f_MHz, pol=ms.beacon_pol, compscans='holo,track',scans='track', ant=ds.obs_params['track_ants'].split(',')[0])
    katselib.plot_signalpol(ds, f_MHz=ms.f_MHz, pol=ms.beacon_pol, compscans='holo,track',scans='track', ant=ant)
    plt.title(ms.tags[-1], loc="left")

In [ ]:
# Find timingoffsets to use for these datasets. This may require some iteration!
for ms in ms_Ku0:
    hologreport.check_timingoffset(cached_url(ms.fid), ms.f_MHz, ant, timingoffset=[0,0.02,0.04,0.08], extent=None)

In [ ]:
# Double-check the final value before loading the data
# - choose the smallest delta that increases amplitude SNR significantly AND doesn't increasing phase RMS significantly
for ms, t_o in zip([0,0.1,0,0], ms_Ku0):
    if (t_o != 0):
        hologreport.check_timingoffset(cached_url(ms.fid), ms.f_MHz, ant, timingoffset=[t_o-0.02,t_o,t_o+0.02], extent=0.5)

In [ ]:
### Load the data
load_extent = np.inf # Load the max extent defined for each record

if True: # Beacon signals
    dMHz = 0.1 # The default, selects 1 channel i.e. highest SNR on beacon (for Ku,S,L & UHF bands)
    # load_extent = predK.clipextent
    datasets = ms_Ku0
else: # S-band continuum
    dMHz = 30 # i.e. ~1% fractional BW in S4 and at most 1.7% in S0
    datasets = ms_Sband0


# cached_url = lambda cbid: f"./l1_data/{cbid}/{cbid}_sdp_l0.full.rdb"
# Use the above e.g in case it is not available in the archive anymore but cached locally
cached_url = util.cbid2url
for ms in datasets:
    # ms.beams.clear(); ms.apmapsH.clear(); ms.apmapsV.clear() # Un-comment to reload all
    if (len(ms.beams) > 0): continue
    # ds = util.open_dataset(ms.fid, cache_root="./l1_data") # Un-comment to cache locally
    b, aH, aV = hologreport.load_data(cached_url(ms.fid), ms.f_MHz, ant, DISHPARAMS, polswap=ms.polswap, timingoffset=ms.timingoffset,
                                      clipextent=min(ms.clipextent,load_extent), loadscan_cycles=ms.cycles, overlap_cycles=ms.overlap_cycles,
                                      findwrap="findwrap" in ms.tags, extralmoffset='auto' if "extralmoffset=auto" in ms.tags else [0,0],
                                      dMHz=dMHz, gridsize=512, flag_slew=True, flags_hrs=ms.flags_hrs, ignoreantennas=ms.ignoreantennas)
    ms.beams.extend(b); ms.apmapsH.extend(aH); ms.apmapsV.extend(aV)

### Quick Look at Maps

In [ ]:
# Quick look at the continuum datasets
# Note: Third sidelobe should be clearly defined in order to make a "good enough" map
for ms in ms_Sband0:
    plt.figure(figsize=(14,7))
    for i,bm in enumerate(ms.beams):
        plt.subplot(1,len(ms.beams),i+1); bm.plot("Gx", doclf=False)

In [ ]:
# Plot all FIk# results
npol = 1
for fi, ms_ in enumerate([ms_Ku0, ms_Ku1, ms_Ku2, ms_Ku3, ms_Ku4[:-1], ms_Ku4[-1:]]):
    if (fi > 4): fi -= 1 # Hack for the way i'm mapping this
    axs = plt.subplots(npol,4, layout='tight', figsize=(18,4*npol), squeeze=False)[1]
    plt.suptitle("FIk#%d"%fi)
    ii = [0,1,2,3] # 4 elevation brackets
    for ms in ms_[::-1]: # If there ar duplicates in elevation, use the later ones first (later ones better quality?)
        bm = ms.beams[0] # Only first frequency, if there are multiple
        el = bm.el_deg
        i = 0 if (el < 25) else (1 if el < 40 else (2 if el < 45 else 3))
        if (i in ii):
            ii.pop(ii.index(i))
            # H-pol feed
            ax = axs[0][i]
            plt.sca(ax); bm.plot("Gx", clim=[10, -60], doclf=False); ax.set_xlim(-1,1); ax.set_ylim(-1,1)
            ax.set_title("%d, FI %.2fdeg\n %s @ %.fdegEl" % (ms.fid, bm.feedindexer_deg[1], bm.dataset.target.name, el), size=6)
            # V-pol feed
            if (len(axs) == 2):
                ax = axs[1][i]
                plt.sca(ax); bm.plot("Gy", clim=[10, -60], doclf=False); ax.set_xlim(-1,1); ax.set_ylim(-1,1)
                ax.set_title("%d, FI %.2fdeg\n%s @ %.fdegEl" % (ms.fid, bm.feedindexer_deg[1], bm.dataset.target.name, el), size=7)
            _im = ax.get_children()[0]
    # Also make un-used axes consistent
    for i in ii:
        ax = axs[0][i]; plt.colorbar(_im, ax=ax); ax.set_aspect('equal')

## Analysis for FI Angle Determination

In [ ]:
ku = hologreport.generate_results(ms_Ku0, predK, DF=8, eb_extent=(-0.3,0.3),
                                    SNR_min=30, phaseRMS_max=50, cmap=CM, makepdfs=False)

In [ ]:
# Combine and filter as necessary
# _ku_ = hologreport.collate_results(ku, ku_) # Combine beacons & continuum
# _ku_ = hologreport.filter_results(_ku_, exclude_tags=["HISPASAT 36W-1"], enviro_filter=lambda enviro: max(enviro['wind_mps'])<=5)

In [ ]:
hologreport.plot_offsets_el([ku["FIk0"]], ["FIk0"], fit="theil-sen")

In [ ]:
hologreport.plot_offsets_freq([ku["FIk0"]], ["Ku-Band, night & day"])

In [ ]:
s = hologreport.generate_results(ms_Sband0, predS, eb_extent=(-1.2,1.2),
                                    SNR_min=20, phaseRMS_max=30, cmap=CM, makepdfs=False)

In [ ]:
hologreport.plot_offsets_el([s["FIs0"]], ["FIs0"], fit="theil-sen")

In [ ]:
hologreport.plot_offsets_freq([s["FIs0"]], ["S-Band, night & day"])

In [ ]:
_,ku_Y_f,_ = hologreport.plot_offsets_el([ku["FI1"]], ["FI1"], fit="lin", elspec_deg=[20,50])[0]

In [ ]:
_,s_Y_f,_ = hologreport.plot_offsets_el([s["FI1"]], ["FI1"], fit="lin", elspec_deg=[20,50])[0]

In [ ]:
_,l_Y_f,_ = hologreport.plot_offsets_el([l["FI1"]], ["FI1"], fit="lin", elspec_deg=[20,50])[0]

In [ ]:
print("[20,50]degEl :", ku_Y_f)
print("[20,50]degEl :", "TODO")#s_Y_f)
print("[20,50]degEl :", l_Y_f)

## Calculate Antenna (Aperture) Efficiency from holography maps

NB: This must be done on datasets with the FI angles used for the actual tests!

In [ ]:
from analysis import katsemodels as models

In [ ]:
katselib.ls_archive(f"Antennas:*{ant}* AND StartTime:[2025-05-20T0:0:0Z TO *] AND InstructionSet:\"scan-extent 24*\" AND InstructionSet:*holography*",
                    min_duration=1800, fields=["CaptureBlockId","StartTime","CenterFrequency",'Description','InstructionSet'], field_len=240);

In [ ]:
# L-band continuum
ms_L = [#predL, # Should have been loaded with clipextent > 20deg!
        hologreport.ResultSet(1752164297,f_MHz=np.linspace(900,1710,17),beacon_pol=None,clipextent=24.0,tags=["FIl2","night","3C 273"]),
        hologreport.ResultSet(1752335861,f_MHz=np.linspace(900,1710,17),beacon_pol=None,clipextent=24.0,tags=["FIl1","day","3C 273"]), # RFI! Final FI - result same as FIl2
       ]

dMHz = 10 # ~1% fractional BW
for ms in ms_L:
    ds = util.open_dataset(ms.fid, cache_root="./l1_data") # Ensure it is cached locally
    b, aH, aV = hologreport.load_data(cached_url(ms.fid), ms.f_MHz, ant, DISHPARAMS, clipextent=ms.clipextent, loadscan_cycles=ms.cycles,
                                      dMHz=dMHz, gridsize=512, flag_slew=True, flags_hrs=ms.flags_hrs)
    ms.beams.extend(b); ms.apmapsH.extend(aH); ms.apmapsV.extend(aV)

In [ ]:
# Compare results & compare to the model generated on MKE119 (../ska119/MKE119-SetToWork.ipynb#Calculate-Antenna-(Aperture)-Efficiency-from-holography-maps)
# Note: 15.3m is the geometric equivalent diameter of the main reflector plus the top wings, treated as constant for "efficiency"
plt.figure(figsize=(14,6))
for i,ms in enumerate(ms_L):
    ill,ap,Ag = hologreport.recalc_eff(ms.apmapsH,ms.apmapsV,ms.f_MHz, D=15.3, save_to="./l2_data", band="SKA_B2.%d"%i)
    plt.plot(ms.f_MHz, ap[:,0], '.-', label='H #%d'%i, alpha=0.5)
    plt.plot(ms.f_MHz, ap[:,1], '.--', label='V #%d'%i, alpha=0.5)
plt.ylim(84,91) # In case there's RFI...
plt.title("Efficiency(D=15.3m) from predicted patterns & patterns measured at %ddegEl"%ms.beams[0].el_deg)
plt.ylabel("$\eta_{ap}$ [%]"); plt.xlabel("f [MHz]")

# plt.plot(ms.f_MHz, models.get_ant_eff(ms.f_MHz, pol="H", band="SKA_B2"), 'k-', label="Model H")
# plt.plot(ms.f_MHz, models.get_ant_eff(ms.f_MHz, pol="V", band="SKA_B2"), 'k--', label="Model V")
# Explicitly get the predicted models
!ls -ln {models.aperture_efficiency_dir}/SKA_B2_ant_eff.csv
model = models.Ap_Eff(band="SKA_B2")
plt.plot(ms.f_MHz, model(ms.f_MHz, "H"), 'k-', label="Model H") # The model was scaled for 15.3m
plt.plot(ms.f_MHz, model(ms.f_MHz, "V"), 'k--', label="Model V")
plt.legend(); plt.grid(True)

In [ ]:
katselib.ls_archive(f"Antennas:*{ant}* AND StartTime:[2025-05-20T0:0:0Z TO *] AND InstructionSet:\"scan-extent 13*\" AND InstructionSet:*holography*",
                    min_duration=1800, fields=["CaptureBlockId","StartTime","CenterFrequency",'Description','InstructionSet'], field_len=240);

katselib.ls_archive(f"Antennas:*{ant}* AND StartTime:[2025-05-20T0:0:0Z TO *] AND InstructionSet:\"scan-extent 9*\" AND InstructionSet:*holography*",
                    min_duration=1800, fields=["CaptureBlockId","StartTime","CenterFrequency",'Description','InstructionSet'], field_len=240);

In [ ]:
# S-band continuum
ms_S = [hologreport.ResultSet(1752764292,f_MHz=np.linspace(1760,2610,17),beacon_pol=None,clipextent=13.0,tags=["FIs2","day","3C 279"]),
        hologreport.ResultSet(1752860746,f_MHz=np.linspace(1760,2610,17),beacon_pol=None,clipextent=13.0,tags=["FIs2","night","PKS J1924-2914"]),

        hologreport.ResultSet(1752876066,f_MHz=np.linspace(2640,3490,40),beacon_pol=None,clipextent=9.0,tags=["FIs2","night","PKS 1934-63"]),
       ]
dMHz = [20]*2 + [28]*1 # ~1% fractional BW

for dMHz,ms in zip(dMHz, ms_S):
    ds = util.open_dataset(ms.fid, cache_root="./l1_data") # Ensure it is cached locally
    b, aH, aV = hologreport.load_data(cached_url(ms.fid), ms.f_MHz, ant, DISHPARAMS, clipextent=ms.clipextent, loadscan_cycles=ms.cycles,
                                      dMHz=dMHz, gridsize=512, flag_slew=True, flags_hrs=ms.flags_hrs)
    ms.beams.extend(b); ms.apmapsH.extend(aH); ms.apmapsV.extend(aV)

In [ ]:
# Compare results & save new model.
# Note: 15.3m is the geometric equivalent diameter of the main reflector plus the top wings, treated as constant for "efficiency"
plt.figure(figsize=(14,6))
for i,ms in enumerate(ms_S):
    ill,ap,Ag = hologreport.recalc_eff(ms.apmapsH,ms.apmapsV,ms.f_MHz, D=15.3, save_to="./l2_data", band="SKA_S.%d"%i)
    plt.plot(ms.f_MHz, ap[:,0], '.-', label='H #%d'%i, alpha=0.5)
    plt.plot(ms.f_MHz, ap[:,1], '.-', label='V #%d'%i, alpha=0.5)
plt.title("Efficiency(D=15.3m) from predicted patterns & patterns measured at %ddegEl"%ms.beams[0].el_deg)
plt.ylabel("$\eta_{ap}$ [%]"); plt.xlabel("f [MHz]")

f_MHz = np.linspace(1750,3500,100)
plt.plot(f_MHz, models.get_ant_eff(f_MHz, pol="H", band="SKA_S"), 'k-', label="Model H")
plt.plot(f_MHz, models.get_ant_eff(f_MHz, pol="V", band="SKA_S"), 'k--', label="Model V")
plt.legend(); plt.grid(True)

In [ ]:
# Combine S0 & S2 (covers the whole S-band without gaps or discontinuity!)
## Don't use the first dataset - seems anomalous at 1760MHz?
!cp ./l2_data/SKA_S.1_ant_eff.csv ./l2_data/SKA_S_ant_eff.csv
!tail -n +4 ./l2_data/SKA_S.2_ant_eff.csv >> ./l2_data/SKA_S_ant_eff.csv

# Feed Indexer Adjustments

The initial angles coded into the ACU must be refined from RF measurements.

The relative angles should match what's in MKE-316-040000-ODC-TR-0145_#TBD (Feed Indexer FAT).

Our approach is to use RF measured results to derive the correct FI angles for one band, then the others should follow from the relative angles above. Holography Y_f and poinitng P4 are closely related, but may differ systematically (similarly for all bands) due to misalignment of the subreflector that can be compensated by offsetting the feed. We adjust to minimize Y_f (lowest beam distortion) rather than P4 (just boresight direction).

The updated angles are written into [Customer_Config.json](https://github.com/ska-sa/katconfig/tree/karoo-dvs/static/acu-parameters) and then pushed out to the ACU to take effect.

In [ ]:
dms2deg = lambda d,m,s: abs(d)+abs(m)/60+abs(s)/3600

def asec2dms(asec):
    d = np.trunc(asec/3600) # Signed float
    asec = abs(asec - d*3600) # Lose the sign
    m = int(asec/60)
    s = asec - m*60
    return f"{d:+03.0f}:{m:02d}:{s:02.1f}"

# Values from MKE-316-040000-ODC-TR-0145_#TBD (Feed Indexer FAT):
FI_factory = np.r_[103.399,-18.61,-100.0]

In [ ]:
### Initial adjustments (Customer_Config.json on 30/01/2026)
current_FI = np.r_[103.37, -18.62, -99.97] # POS:1, 7, 2 = Ku, S, B2

# Ku-band on 7/02/2026, from first series of holography
dP4, dFI_angle = util.calc_FIangle_adjustment(delta_Yf=np.r_[37.3, 0, 0]) # Ku, S, B2
print("From holog: suggested new FI angles from 'current':", current_FI + dFI_angle)
print("From holog: suggested new FI angles from 'FAT':", FI_factory+(current_FI[0]-FI_factory[0]) + dFI_angle)
print("    (equivalent P4 delta to add to current P4:", [asec2dms(_*3600) for _ in dP4], ")")

In [ ]:
### Subsequent adjustment (Customer_Config.json on 39/01/2026)
# B2 on 25/07/2025
current_FI = np.r_[103.296, -18.621, -99.97] # POS:1, 7, 2 = Ku, S, B2
dP4, dFI_angle = util.calc_FIangle_adjustment(delta_Yf=np.r_[0, 0, 13.1]) # Ku, S, B2
print("From holog: suggested new FI angles from 'current':", current_FI + dFI_angle)
print("From holog: suggested new FI angles from 'FAT':", FI_factory+(current_FI[0]-FI_factory[0]) + dFI_angle)
print("    (equivalent P4 delta to add to current P4:", [asec2dms(_*3600) for _ in dP4], ")")